In [2]:
import pandas as pd
import ast
import numpy as np
import sys
import geopandas
from geopy.distance import geodesic
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points
from shapely import wkt

### import data

In [5]:
properati = pd.read_csv('properati_data/combined_sell.csv', encoding= "ISO-8859-1")
prop_data = pd.read_csv('prop_data/cleaned_data.csv', encoding= "ISO-8859-1")
census = pd.read_csv('census/cleaned_data2.csv',encoding= "ISO-8859-1")

### Create GeoPandas dataframes

In [6]:
def modify_poly(data):
    coord_list = []
    #row = ast.literal_eval(data)
    for coord in data:
        coord_list.append(tuple(coord))
    return coord_list

In [7]:
# create a new column called coordinates on properati data frame
properati['coordinates'] = list(zip(properati.lon, properati.lat))
properati['coordinates'] = properati['coordinates'].apply(Point)

# convert properati data into geopandas dataframe
properati_gdf = geopandas.GeoDataFrame(properati, geometry='coordinates')

# convert the longitudes and latitudes in the properaty values dataframe into float (they were originally in string)
prop_data.Latitude = pd.to_numeric(prop_data.Latitude, errors='coerce')
prop_data.Longitude = pd.to_numeric(prop_data.Longitude, errors='coerce')

#print (properati_gdf.coordinates.head(5))
print (properati_gdf.coordinates.isnull().values.any())
print (len(properati_gdf))

# convert property values data into geopandas dataframe
prop_data['coordinates'] = list(zip(prop_data.Longitude, prop_data.Latitude))
#prop_data['coordinates'] = list(zip(prop_data.Latitude, prop_data.Longitude))
prop_data['coordinates'] = prop_data['coordinates'].apply(Point)
prop_data_gdf = geopandas.GeoDataFrame(prop_data, geometry='coordinates')


# convert census polygon data into tuples and polygon
census.coordinates = census.coordinates.apply(lambda x: ast.literal_eval(x)[0])
census.coordinates = census.coordinates.apply(lambda x: modify_poly(x))
census.coordinates = census.coordinates.apply(Polygon)

census_gdf = geopandas.GeoDataFrame(census, geometry='coordinates')
print (len(census_gdf))
#print (census_gdf.coordinates[0])
print (census_gdf.coordinates.isnull().values.any())

False
1871732
3552
False


### Filter only data that is in the city of the Buenos Aires

In [8]:
properati_census_gdf = geopandas.sjoin(properati_gdf, census_gdf, how="inner", op='within')
print ('{:.2f}%'.format(len(properati_census_gdf)/len(properati_gdf)))

print (len(properati_census_gdf))

/anaconda3/envs/geopandas/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


0.25%
464869


#### save the properati/census combined data because the sjoin takes forever

In [3]:
temp_df = pd.DataFrame(properati_census_gdf)
temp_df.to_csv('properati_data/sell_census.csv')
temp_df = temp_df.drop_duplicates(subset='id_left')
print (prop_census_df.head(5))
properati_census_gdf = geopandas.GeoDataFrame(prop_census_df, geometry='coordinates')

NameError: name 'properati_census_gdf' is not defined

In [34]:
temp_df = pd.read_csv('properati_data/sell_census.csv')
#print (temp_df.id_left.head(5))
temp_df = temp_df.drop_duplicates(subset='id_left')
temp_df.coordinates = list(zip(temp_df.lon, temp_df.lat))
temp_df.coordinates = temp_df.coordinates.apply(Point)
properati_census_gdf = geopandas.GeoDataFrame(temp_df, geometry='coordinates')
#print ('{:.2f}'.format((len(properati_census_gdf)/ 464869)*100))

#### import barrios geojson

In [35]:
barrios = pd.read_csv('shape files/barrios.csv')
barrios.WKT = barrios.WKT.apply(wkt.loads)

barrio_df = pd.read_csv('barrio_table.csv')
barrio_df['b_id'] = [int(barrio_df[barrio_df.Barrio == i].id) for i in barrio_df.Barrio]
barrio_df['barrio'] = barrio_df.Barrio

barrios = pd.merge(barrios, barrio_df, on='barrio', how='left')
print (barrios.head(5))

barrios_gdf = geopandas.GeoDataFrame(barrios, geometry='WKT')

                                                 WKT            barrio  \
0  POLYGON ((-58.4528200492791 -34.5959886570639,...         CHACARITA   
1  POLYGON ((-58.4655768128541 -34.5965577078058,...          PATERNAL   
2  POLYGON ((-58.4237529813037 -34.5978273383243,...      VILLA CRESPO   
3  POLYGON ((-58.4946097568899 -34.6148652395239,...  VILLA DEL PARQUE   
4  POLYGON ((-58.4128700313089 -34.6141162515854,...           ALMAGRO   

   comuna    perimetro          area  Unnamed: 0            Barrio    id  b_id  
0      15  7725.695228  3.118101e+06        26.0         CHACARITA  26.0  26.0  
1      15  7087.513295  2.229829e+06        27.0          PATERNAL  27.0  27.0  
2      15  8132.699348  3.613584e+06         6.0      VILLA CRESPO   6.0   6.0  
3      11  7705.389797  3.399596e+06        29.0  VILLA DEL PARQUE  29.0  29.0  
4       5  8537.901368  4.050752e+06        12.0           ALMAGRO  12.0  12.0  


In [36]:
properati_census_gdf = properati_census_gdf.drop(columns=['id_left', 'id_right', 'index_right'], axis=1)
print (properati_census_gdf.columns.values)
final_gdf = geopandas.sjoin(properati_census_gdf , barrios_gdf, how="inner", op='within')
print (final_gdf.columns.values)
print (len(final_gdf))


['Unnamed: 0' 'dataset_date' 'created_on' 'operation' 'property_type'
 'place_name' 'country_name' 'state_name' 'geonames_id' 'lat_lon' 'lat'
 'lon' 'price' 'currency' 'price_aprox_local_currency' 'price_aprox_usd'
 'coordinates' 'AREA' 'Commune' 'DEPTO' 'FRAC' 'Computer Percent'
 'Computer Quantile' 'Cellular Percent' 'Cellular Quantile' 'Rent Percent'
 'Rent Quantile' 'LINK' 'Immigration Percent' 'Immigration Quantile'
 'Education Percent' 'Education Quantile' 'PERIMETER' 'Owner Percent'
 'Owner Quantile' 'PROV' 'RADIO' 'RADPAIS_' 'REDCODE' 'Regular Percent'
 'Regular Quantile' 'TIPO' 'Uninhabited Percent' 'Uninhabited Quantile'
 'type']
['Unnamed: 0_left' 'dataset_date' 'created_on' 'operation' 'property_type'
 'place_name' 'country_name' 'state_name' 'geonames_id' 'lat_lon' 'lat'
 'lon' 'price' 'currency' 'price_aprox_local_currency' 'price_aprox_usd'
 'coordinates' 'AREA' 'Commune' 'DEPTO' 'FRAC' 'Computer Percent'
 'Computer Quantile' 'Cellular Percent' 'Cellular Quantile' 'Rent 

In [37]:
filtered_gdf = final_gdf[['dataset_date', 'created_on', 'operation', 'property_type', 'place_name',\
                          'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',\
                         'Commune', 'barrio','b_id', 'comuna', 'perimetro','area']]

filter_df = pd.DataFrame(filtered_gdf)
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

In [43]:
barrios_filter_df = filter_df[['barrio','comuna','b_id', 'perimetro','area','price_aprox_local_currency', 'price_aprox_usd','created_on']]
grouped = barrios_filter_df.groupby(['b_id','barrio'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"})
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
print (grouped.head(5))
grouped.to_csv('properati_data/properati_sell_grouped.csv')

   b_id_        barrio_  price_aprox_local_currency_min  \
0    0.0        PALERMO                        85497.90   
1    1.0       BELGRANO                        92550.18   
2    2.0          NUÑEZ                        77468.62   
3    3.0  VILLA ORTUZAR                       798425.00   
4    4.0      BALVANERA                       101373.93   

   price_aprox_local_currency_max  price_aprox_local_currency_mean  \
0                     270456790.5                     6.432754e+06   
1                     107992500.0                     6.114209e+06   
2                     206644442.4                     4.963615e+06   
3                     114435750.0                     4.264157e+06   
4                      86372500.0                     2.711736e+06   

   price_aprox_local_currency_std  price_aprox_usd_min  price_aprox_usd_max  \
0                    8.941423e+06              5354.16           15692300.0   
1                    7.887623e+06              5256.89            

In [44]:
grouped.to_json("properati_data/properati_sell.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)



In [40]:
filter_df['created_on'] = pd.to_datetime(filter_df['created_on'])
filter_df['month'] = filter_df.created_on.apply(lambda x: x.month)
filter_df['year'] = filter_df.created_on.apply(lambda x: x.year)
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

barrios_filter_df = filter_df[['barrio','b_id','dataset_date','comuna','perimetro','area','price_aprox_local_currency', 'price_aprox_usd',\
                             'created_on', 'month', 'year']]
print (barrios_filter_df.head(5))

     barrio  b_id  dataset_date  comuna     perimetro          area  \
0  BARRACAS  32.0        201501       4  13018.210271  7.961000e+06   
1  BARRACAS  32.0        201501       4  13018.210271  7.961000e+06   
2  BARRACAS  32.0        201502       4  13018.210271  7.961000e+06   
3  BARRACAS  32.0        201502       4  13018.210271  7.961000e+06   
4  BARRACAS  32.0        201502       4  13018.210271  7.961000e+06   

   price_aprox_local_currency  price_aprox_usd created_on  month  year  
0                  3109410.00        180000.00 2014-08-08      8  2014  
1                  2591175.00        150000.00 2014-08-08      8  2014  
2                  1627587.50         95000.00 2015-11-27     11  2015  
3                    85275.81          4977.43 2015-11-28     11  2015  
4                  2826862.50        165000.00 2015-11-10     11  2015  


In [41]:
grouped_month = barrios_filter_df.groupby(['barrio', 'b_id','month', 'year'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"}).fillna(0) #for the std deviation values that resulted in NaN 
grouped_month.columns = ["_".join(x) for x in grouped_month.columns.ravel()]
print (grouped_month.head(5))
print (len(grouped_month))

     barrio_  b_id_  month_  year_  price_aprox_local_currency_min  \
0  AGRONOMIA   25.0       1   2015                      1347411.00   
1  AGRONOMIA   25.0       1   2016                      1037952.50   
2  AGRONOMIA   25.0       1   2017                      1916220.00   
3  AGRONOMIA   25.0       1   2018                      1366497.82   
4  AGRONOMIA   25.0       2   2015                      1209215.00   

   price_aprox_local_currency_max  price_aprox_local_currency_mean  \
0                       1347411.0                     1.347411e+06   
1                       1437165.0                     1.237559e+06   
2                       4774581.5                     2.573210e+06   
3                      11388300.0                     4.839287e+06   
4                       1209215.0                     1.209215e+06   

   price_aprox_local_currency_std  price_aprox_usd_min  price_aprox_usd_max  \
0                    0.000000e+00              78000.0              78000.0   


In [42]:
grouped_month.to_csv('properati_data/properati_sell_grouped_month.csv')
grouped_month.to_json("properati_data/properati_sell_month.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

